In [2]:
import numpy as np
import pickle as pickle
import pandas as pd
import math
import pyeeg as pe
from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

import os
#import tensorflow as tf
import time

In [3]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
#List of subjects

In [15]:
def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open('./data_preprocessed_python/data_preprocessed_python\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0;
            c=0
            while start + window_size < data.shape[1]:
                c=c+1
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
                
        meta = np.array(meta)
        #np.save('s' + sub, meta, allow_pickle=True, fix_imports=True)

def testing (M, L, model):
    '''
    arguments:  M: testing dataset
                L: testing dataset label
                model: scikit-learn model

    return:     void
    '''
    output = model.predict(M[0:78080:32])
    label = L[0:78080:32]

    k = 0
    l = 0

    for i in range(len(label)):
        k = k + (output[i] - label[i])*(output[i] - label[i]) #square difference 

        #a good guess
        if (output[i] > 5 and label[i] > 5):
            l = l + 1
        elif (output[i] < 5 and label[i] <5):
            l = l + 1

    print ("l2 error:", k/len(label), "classification accuracy:", l / len(label),l, len(label))

In [ ]:
for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)

In [4]:
data_training = []
label_training = []
data_testing = []
label_testing = []
data_validation = []
label_validation = []

for subjects in subjectList:

    with open('s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file,allow_pickle=True)
        print(sub.shape)
        for i in range (0,sub.shape[0]):
            if i % 8 == 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            elif i % 8 == 1:
                data_validation.append(sub[i][0])
                label_validation.append(sub[i][1])
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

np.save('data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save('data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

np.save('data_validation', np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save('label_validation', np.array(label_validation), allow_pickle=True, fix_imports=True)
print("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)

(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
training dataset: (468480, 70) (468480, 4)
testing dataset: (78080, 70) (78080, 4)
validation dataset: (78080, 70) (78080, 4)


In [6]:
with open('data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)
    
with open('label_training.npy', 'rb') as fileTrainL:
    Y  = np.load(fileTrainL)
    
X = normalize(X)
Z = np.ravel(Y[:, [1]])

Arousal_Train = np.ravel(Y[:, [0]])
Valence_Train = np.ravel(Y[:, [1]])
Domain_Train = np.ravel(Y[:, [2]])
Like_Train = np.ravel(Y[:, [3]])



with open('data_validation.npy', 'rb') as fileTrain:
    M  = np.load(fileTrain)
    
with open('label_validation.npy', 'rb') as fileTrainL:
    N  = np.load(fileTrainL)

M = normalize(M)
L = np.ravel(N[:, [1]])

Arousal_Test = np.ravel(N[:, [0]])
Valence_Test = np.ravel(N[:, [1]])
Domain_Test = np.ravel(N[:, [2]])
Like_Test = np.ravel(N[:, [3]])

In [7]:
clf = svm.SVR()
clf.fit(X[0:468480:32], Z[0:468480:32])

C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [19]:
Lik_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Lik_R.fit(X[0:468480:32], Like_Train[0:468480:32])
testing (M, Like_Test, Lik_R)

l2 error: 2.4788304602635534 classification accuracy: 0.8471311475409836 2067 2440



import numpy as np
f=np.load('s01.npy',allow_pickle=True)
print(f.shape)
print(f[19519][0].shape)
meta=[]

with open('./data_preprocessed_python/data_preprocessed_python\s' + '01' + '.dat', 'rb') as file:
        c=0
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0;
            while start + window_size < data.shape[1]:
                if i==0:
                    c=c+1
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
        
                    Y = pe.bin_power(X, band, sample_rate)#FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
print(c)
print(meta[1])  
print(meta[2])
print(meta[487])

In [4]:
with open('data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)
np.savetxt("datatrain.csv",X)

In [2]:
import numpy as np
with open('data_testing.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)
np.savetxt('datatest.csv',X)

In [8]:
import numpy as np
with open('label_testing.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)
np.savetxt('labeltest.csv',X)
np.shape(X[: ,0:2])
np.savetxt('labeltest1.csv',X[:,0:2])